<a href="https://colab.research.google.com/github/chandrusuresh/ReinforcementLearning/blob/master/Policy_Iteration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from scipy.special import factorial

##Policy Iteration
Policy iteration is an algorithm for evaluating the value function and improving it in successive iterations. Each iteration involves 2 steps - first step evaluates the policy and the second improves it. The first step is the same as the iterative policy evaluation algorithm and second step is the iteration policy improvement algorithm. Refer to the [Grid World notebook](https://github.com/chandrusuresh/ReinforcementLearning/blob/master/GridWorld.ipynb) for more details on these two steps/algorithms.

The state and action value function equations are summarized here:

$$\textbf{State Value Function:  }  v_{k+1}(s) = \sum_a{\pi(a|s)} \sum_{a,s'}{p(s',r|a,s) \left(r + \gamma v_k (s')\right)}  $$

$$\textbf{Action Value Function:  }  q_\pi (s,a) = \sum_{r,s'}{p(s',r|a,s) \left(r + \gamma \sum_{a'}{\pi(a'|s') q_\pi (s',a')}\right)}  $$

## Jack's Rental Car Problem

Jack manages two locations for a nationwide car
rental company. Each day, some number of customers arrive at each location to rent cars.

Cost of rental = \$10

Cost of moving cars = -$2

Max Cars = 20

Max cars moved = 5

Number of cars rented/returned is a Poisson distribution given by,
$$P(rented/returned = n) = \frac{\lambda^n}{n!} e^{-\lambda}$$ where $\lambda$ is the expected number.

$\lambda_{rent}^1 = 3$, $\lambda_{rent}^2 = 4$ and

$\lambda_{return}^1 = 3$,$\lambda_{return}^2 = 2$

$gamma = 0.9$
Time steps = days

State variable = Number of cars at each location at the end of the day

Action variable = Number of cars moved

## Solution
The state vector, $s_t = \left[\begin{matrix}s_1 \\ s_2 \end{matrix}\right]$ where $s_i$ is the number of cars in location $i$ at the beginning of day $t$

The action vector, $a_t$ is the number of cars moved between locations 1 & 2 at the end of day $t$

The transition probability, $p_{trans} = p(r,s_{t+1}|s_t,a_t)$

Assuming the rentals & returns are independent of each other and location, we have:

$$\begin{align*} p_{trans,i} &= p(r,s_{t+1,i}|s_{t,i},a_t) \\ 
&= p(rent=x,retn=y|s_{t,i},a_t) \\
 &= p(retn=y|rent=x)\times p(rent=x) \end{align*}$$ where $s_{t+1} = min(20,max(0,s_t-x+y)) $


In [0]:
max_cars = 20
max_move = 5
rent = [3,4]
retn = [3,2]
rent_cost = 10
move_cost = -2
gamma = 0.9

def poisson(lamda,n):
  return lamda**n/factorial(n)*np.exp(-lamda)

def prob_rent(init_cars,lambda_rent):
  sum_prob = 0
  prob_rent = dict()
  for i in range(init_cars):
    prob_rent[i] = poisson(lambda_rent,i)
    sum_prob = sum_prob + prob_rent[i]
  prob_rent[init_cars] = 1 - sum_prob
  return prob_rent

def prob_retn(init,prob_rent,lambda_retn):
  prob_cars = dict()
  for k in prob_rent.keys():
    c = init-k
    sum_prob = 0
    for i in range(max_cars-c):
      prob = poisson(lambda_retn,i)
      sum_prob = sum_prob + prob
      if c+i in prob_cars.keys():
        prob_cars[c+i] = prob_cars[c+i] + prob_rent[k]*prob
      else:
        prob_cars[c+i] = prob_rent[k]*prob
    if max_cars in prob_cars.keys():
      prob_cars[max_cars] = prob_cars[max_cars] + prob_rent[k]*(1-sum_prob)
    else:
      prob_cars[max_cars] = prob_rent[k]*(1-sum_prob)
  return prob_cars

def transition(lambda_rent,lambda_retn):
  p_trans = dict()
  for init_cars in range(max_cars+1):
    p_rent = prob_rent(init_cars,lambda_rent)
    p_trans[init_cars] = prob_retn(init_cars,p_rent,lambda_retn)
  return p_trans

p_trans_1 = transition(rent[0],retn[0])
p_trans_2 = transition(rent[1],retn[1])

### Initialize State Value Fuction and Policy

In [0]:
state_value = np.zeros((max_cars+1,max_cars+1))
actions = list(range(-max_move,max_move+1))
# pi = np.zeros((2*max_move+1,max_cars+1,max_cars+1))
# for j in range(pi.shape[2]):
#   for i in range(pi.shape[1]):
#     for k in range(max_move+1):
#       if k == 0:
#         pi[max_move,i,j] = 1
#       else:
#         if k <= i:
#           pi[max_move+k,i,j] = 1
#         if k <= j:
#           pi[max_move-k,i,j] = 1
# ## Normalize pi
# for i in range(pi.shape[1]):
#   for j in range(pi.shape[2]):
#     pi[:,i,j] = pi[:,i,j]/np.sum(pi[:,i,j])

### Policy Evaluation



In [0]:
def evaluate_policy(v_s,pi):
  vs_new = np.copy(v_s)
  for i in range(max_cars+1):
    for j in range(max_cars+1):
      vs = state_value[i,j]
      val_sum = 0
      for a in range(pi.shape[0]):
        if pi[a,i,j] == 0:
          continue
        # new_i = i
        # new_j = j
        new_i = min(max_cars,i - actions[a])
        new_j = min(max_cars,j + actions[a])
        reward = move_cost*np.abs(actions[a])
        trans_prob_sum = 0
        for ii in range(max_cars+1):
          p_ii = p_trans_1[new_i][ii]
          for jj in range(max_cars+1):
            p_jj = p_trans_2[new_j][jj]
            prob = p_ii*p_jj
            trans_prob_sum = trans_prob_sum + prob*(reward+gamma*v_s[ii,jj])
        val_sum = val_sum + pi[a,i,j]*trans_prob_sum
      vs_new[i,j] = val_sum
  return vs_new

### Policy Improvement

In [297]:
def improve_policy(v_s):
  pi_new = np.zeros((len(actions),v_s.shape[0],v_s.shape[1]))
  for i in range(v_s.shape[0]):
    for j in range(v_s.shape[1]):
      val = dict()
      for a in range(len(actions)):
        new_i = min(max_cars,i - actions[a])
        new_j = min(max_cars,j + actions[a])
        if new_i >= 0 and new_j >= 0:
          if v_s[new_i,new_j] in val.keys():
            val[v_s[new_i,new_j]].append(a)
          else:
            val[v_s[new_i,new_j]] = [a]
      # print(i,j,val)
      max_val = np.max(list(val.keys()))
      for a in val[max_val]:
        pi_new[a,i,j] = 1/float(len(val[max_val]))
  return pi_new
new_pi = improve_policy(state_value)
# del_pi = (new_pi- pi)[:,:,0]
# print(np.round(del_pi,3))
print(np.max(np.abs(new_pi- pi)))

0.0


## Test

In [298]:
cars_init = [7,7]
p_rent = prob_rent(cars_init[1],rent[1])
p_retn = prob_retn(cars_init[1],p_rent,retn[1])
print("Probability of Rent")
print(p_rent)
print(np.sum(list(p_rent.values())))
print("Probability of Return")

total_prob = 0
for i in range(max_cars+1):
  print(i,p_retn[i]-p_trans_2[cars_init[1]][i])
  total_prob = total_prob + p_retn[i]
print(total_prob)
print(np.sum(list(p_retn.values())))

Probability of Rent
{0: 0.01831563888873418, 1: 0.07326255555493671, 2: 0.14652511110987343, 3: 0.19536681481316456, 4: 0.19536681481316456, 5: 0.15629345185053165, 6: 0.1041956345670211, 7: 0.11067397840257387}
1.0
Probability of Return
0 0.0
1 0.0
2 0.0
3 0.0
4 0.0
5 0.0
6 0.0
7 0.0
8 0.0
9 0.0
10 0.0
11 0.0
12 0.0
13 0.0
14 0.0
15 0.0
16 0.0
17 0.0
18 0.0
19 0.0
20 0.0
1.0000000000000002
0.9999999999999999


In [299]:
# print(state_value[0,0])
max_iters = 50
iter = 0
vs_prev = state_value
error = 1E5
convergence = []
while iter < max_iters and error > 0.1:
  pi = improve_policy(vs_prev)
  vs_new = evaluate_policy(vs_prev,pi)
  error = np.max(np.abs(vs_new-vs_prev))
  vs_prev = vs_new
  print(iter,error)
  convergence.append(error)
  iter = iter+1
  
print(np.round(vs_new,1))

0 5.454545454545458
1 9.504054323621
2 10.838609499039983
3 10.16353286253598
4 9.980824825570991
5 9.055050233266016
6 7.697964619782638
7 7.323820124860347
8 7.254568739264418
9 2.807780498027654
10 9.062613651216381
11 7.219012546639682
12 5.549291970007189
13 4.427152613723962
14 6.852099974250972
15 0.6420663662344914
16 0.5230006492801067
17 0.45581487115023833
18 0.39900456068657775
19 0.3498105257255091
20 4.275147274821215
21 2.2153453043628204
22 0.24360425284415754
23 0.21408827118834495
24 2.3379471481149494
25 0.1635192581894067
26 0.14520613292609852
27 0.13019399765464357
28 0.11665397331785954
29 0.10450586583887755
30 0.09365544963666039
[[-21.4 -23.5 -23.5 -25.6 -25.6 -27.9 -28.  -28.3 -30.6 -28.3 -26.2 -29.9
  -30.4 -30.7 -30.8 -30.9 -31.  -31.1 -31.1 -31.2 -31.3]
 [-21.5 -21.5 -23.6 -23.6 -25.9 -26.  -26.3 -28.6 -26.3 -28.2 -27.9 -28.4
  -28.7 -28.8 -28.9 -29.  -29.1 -29.1 -29.2 -29.3 -29.3]
 [-23.5 -21.6 -21.6 -23.9 -24.  -24.3 -26.6 -24.3 -32.8 -25.9 -26.4 -26.7
 

In [305]:
print(pi[:,3,5])

[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]


In [306]:
print(vs_new[4,4])
print(vs_new[3,5])

-22.62972870053331
-24.629728700533285
